First add the import statements needed to pull frames and to use the machine learning model trained earlier

In [3]:
import pandas as pd
import os
import cv2
from roboflow import Roboflow


The general idea is to perform the following:
1. Extract frames from video. This will be a similar process to when we extracted frames from a video earlier to get training data.
2. Instead of storing these images in a separate folder, we will individually pass each image to our machine learning model so to use for classification.
3. Finally, we will print out results to determine if the current path is a paved or unpaved road

First retrieve the API key from the .env file.

In [44]:
!pip install python-dotenv

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
api_key = os.getenv('ROBOFLOW_API_KEY')


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
# Import the trained machine learning model from Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace().project("road-surface-classification-lgxl1")
model = project.version(1).model

print(model)

loading Roboflow workspace...
loading Roboflow project...
{
  "name": "Road Surface Classification",
  "version": "1",
  "base_url": "https://classify.roboflow.com/"
}


In [40]:
# Function to import mp4 video and convert to frames, then save to folder, store list of image names and timestamps in csv

def classify_video_to_frames(video_path):

    # Create dataframe to store image names and timestamps
    df = pd.DataFrame(columns=['image_name', 'timestamp'])
    rows = []

    # Create VideoCapture object
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    # Read until video is completed
    for idx, frame_num in enumerate(range(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), 30)):
        # Set frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:
            # Crop the bottom 20% of the frame
            height, width, _ = frame.shape
            crop_width = int(width * 0.25)  # Width to be cropped on both sides
            cropped_frame = frame[int(height * 0.5):int(height * 0.8), crop_width:-crop_width]


            # Add image name and timestamp to list
            timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)

            # Add image to current folder
            image_name = f'frame{timestamp}.jpg'
            cv2.imwrite("./" + image_name, cropped_frame)

            res = model.predict("./" + image_name).json()

            surface = res["predictions"][0]["predictions"][0]["class"]

            print("The road surface at timestamp {0} is {1}".format(timestamp/1000, surface))

            os.remove(image_name)

    # Release the video capture object
    cap.release()

In [43]:
classify_video_to_frames(video_path='../data/GoPro Video/GX020029.MP4')

The road surface at timestamp 0.0 is Unpaved
The road surface at timestamp 1.0010000000000001 is Unpaved
The road surface at timestamp 2.0020000000000002 is Unpaved
The road surface at timestamp 3.003 is Unpaved
The road surface at timestamp 4.0040000000000004 is Unpaved
The road surface at timestamp 5.005 is Unpaved
The road surface at timestamp 6.006 is Unpaved
The road surface at timestamp 7.007000000000001 is Unpaved
The road surface at timestamp 8.008000000000001 is Unpaved
The road surface at timestamp 9.009 is Unpaved
The road surface at timestamp 10.01 is Unpaved
The road surface at timestamp 11.011000000000001 is Unpaved
The road surface at timestamp 12.012 is Unpaved
The road surface at timestamp 13.013 is Unpaved
The road surface at timestamp 14.014000000000001 is Unpaved
The road surface at timestamp 15.015 is Unpaved
The road surface at timestamp 16.016000000000002 is Unpaved
The road surface at timestamp 17.017 is Unpaved
The road surface at timestamp 18.018 is Unpaved
Th

KeyboardInterrupt: 